# Address scorer
Algorithm to determine whether an address is a potential inside/smart whale

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keys 
import requests
from datetime import datetime, timedelta
import time

In [3]:
address = "0xfa53d837b5ddd616007f91487f041d27edb683a0"

In [4]:
address

'0xfa53d837b5ddd616007f91487f041d27edb683a0'

## Get latest block

In [97]:
def get_latest_block(t=None):
    """gets latest block as an int"""
    if t is None: t = int(time.time())
    request = f"https://api.etherscan.io/api\
                ?module=block\
                &action=getblocknobytime\
                &timestamp={t}\
                &closest=before\
                &apikey={keys.key('etherscan')}".replace(" ", "")
    res = requests.get(request)
    return int(res.json()['result'])

## Panning

In [141]:
address = "0x6c8AdA12895975F541F483Bc155fECFda231aE34"

In [190]:
def get_etherscan(address, tx_type, start=0, end=99999999, size=10000):
    """
    returns the etherscan transactions
    tx_type:
        - "txlist" for normal eth transactions
        - "txlistinternal" for internal transactions
        - "tokentx" for erc20 transactions
        - "tokennfttx" for nft transactions
    """
    page = 1
    etherscan = []
    while True:
        request = f"https://api.etherscan.io/api?module=account\
                    &action={tx_type}\
                    &address={address}\
                    &startblock={start}\
                    &endblock={end}\
                    &page={page}\
                    &offset={size}\
                    &sort=asc\
                    &apikey={keys.key('etherscan')}".replace(" ", "")
        res = requests.get(request).json()["result"]
        etherscan += res
        if len(res)<size: break
        page += 1
    return etherscan

In [231]:
def create_transaction(nor, tok, address):
    """take in two transactions, one normal on token, that happen on same block number"""
    address = address.lower()
    side = "unknown"
    if nor is None: 
        side = "transaction"
    elif tok["to"] == address and nor["from"] == address:
        side = "buy"
    elif tok["from"] == address and nor["to"] == address:
        side = "sell"
    amt = tok["value"]
    symbol = tok["tokenSymbol"]
    block = tok["blockNumber"]
    ts = tok["timeStamp"]
    eth = nor["value"] if nor else 0
    return _create_transaction(block, ts, symbol, side, amt, eth)
def _create_transaction(block, time, symbol, side, amt, eth):
    return {"block":int(block),
            "time":int(time),
            "symbol":symbol,
            "side":side,
            "amt":int(amt),
            "eth":int(eth)}

## TODO:
add transaction hash verification
ie check out: https://etherscan.io/address/0x950aebfd80ca6843b5407fe6472f4b7ca1375a6f#tokentxns
Notice that `0xdbc6dbf365d5124b1aa309a7dbc66fe8906c36235a30beb3f6140c07bef806f6` hash remains the same for multiple transactions and is the same for internal and erc20 tx

Next: iterate through transactions and come up with a score or just separate metrics

Next: Create program to take in something like https://etherscan.io/dex?q=0xa71d0588eaf47f12b13cf8ec750430d21df04974#transactions and search through all of that

In [249]:
def get_transactions(address):
    """returns transactions"""
    normal = get_etherscan(address, "txlist")
    internal = get_etherscan(address, "txlistinternal")
    token = get_etherscan(address, "tokentx")
    itb = list(map(lambda x: int(x["blockNumber"]), internal))
    nob = list(map(lambda x: int(x["blockNumber"]), normal))
    transactions = []
    i = 0
    j = 0
    for t in token:
        tbk = int(t["blockNumber"])
        while i<len(normal)-1 and (nob[i] < tbk or normal[i]["value"] == "0"):
            i += 1
        while j<len(internal)-1 and (itb[j] < tbk or internal[j]["value"] == "0"):
            j += 1
        if nob[i] == tbk:
            transactions.append(create_transaction(normal[i], t, address))
        elif itb[j] == tbk:
            transactions.append(create_transaction(internal[j], t, address))
        else: 
            transactions.append(create_transaction(None, t, address))
    return transactions
        

In [250]:
t = get_transactions("0x950AebFD80ca6843b5407fe6472f4B7Ca1375a6F")

In [256]:
t[::-1][:5]

[{'block': 14711459,
  'time': 1651674054,
  'symbol': 'BAPE',
  'side': 'buy',
  'amt': 1200141612262906,
  'eth': 1000000000000000000},
 {'block': 14711438,
  'time': 1651673764,
  'symbol': 'BAPE',
  'side': 'buy',
  'amt': 1503152012762172,
  'eth': 1200000000000000000},
 {'block': 14706310,
  'time': 1651603374,
  'symbol': 'CULT',
  'side': 'sell',
  'amt': 216937495485970049088231999,
  'eth': 1968287407787497408},
 {'block': 14706310,
  'time': 1651603374,
  'symbol': 'CULT',
  'side': 'sell',
  'amt': 871234921630401803567196,
  'eth': 1968287407787497408},
 {'block': 14706037,
  'time': 1651599830,
  'symbol': 'CULT',
  'side': 'buy',
  'amt': 217808730407600450891799195,
  'eth': 2000000000000000000}]

In [252]:
len(list(filter(lambda x: x["side"] not in ["buy", "transaction", "unknown"], t)))

563